# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
city_weather = pd.read_csv("../WeatherPy/city_weather.csv")
city_weather = city_weather[['City', 'Coordinates', 'Latitude', 'Longitude',
       'Temperature', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country',
       'Date']]
city_weather.head(5)

,City,Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,tuktoyaktuk,"-133.04, 69.45",69.45,-133.04,60.80,82,20,11.41,CA,1598294818
1,port moresby,"147.18, -9.44",-9.44,147.18,71.60,100,40,9.17,PG,1598294819
2,castro,"-50.01, -24.79",-24.79,-50.01,62.40,67,0,7.78,BR,1598294769
3,bodden town,"-81.25, 19.28",19.28,-81.25,79.63,94,90,9.17,KY,1598294819
4,marolambo,"48.12, -20.05",-20.05,48.12,62.02,92,63,1.12,MG,1598294819


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
# Create a list containing coordinates
lat = city_weather["Latitude"]
lon = city_weather["Longitude"]

coordinates = city_weather[["Latitude", "Longitude"]]

#Customize the size of the figure

figure_layout = {
    'width': '750px',
    'height': '650px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable
#markers = gmaps.marker_layer(coordinates)  #, fill_color = "red", stroke_color = "red", scale = 2)

#Add heatmap layer
locations = city_weather[["Latitude", "Longitude"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

# Add the layer to the map
#fig.add_layer(markers)
fig

#coordinates.head(5)

Figure(layout=FigureLayout(border='1px solid black', height='650px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
#Filter for temperatures between 60 & 90 degrees fahrenheit, humidity less than 40%, and wind speed under 10 MPH
narrowed_cities = city_weather.loc[(city_weather["Temperature"] >= 75) & (city_weather["Temperature"] <= 90) & (city_weather["Humidity"] < 40) 
                                   & (city_weather["Wind Speed"] < 10) ,:]

#Print preview of new dataframe
narrowed_cities.head(5)

,City,Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
16,abu samrah,"37.18, 35.3",35.30,37.18,77.32,36,0,6.73,SY,1598294822
70,saint-ambroise,"-71.33, 48.55",48.55,-71.33,77.41,34,40,6.93,CA,1598294836
149,san borja,"-66.85, -14.82",-14.82,-66.85,89.51,36,0,6.73,BO,1598294862
150,baijiantan,"85.18, 45.63",45.63,85.18,82.80,22,85,9.19,CN,1598294862
190,huanta,"-74.25, -12.93",-12.93,-74.25,75.20,23,20,5.82,PE,1598294875


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
hotel_df = narrowed_cities[['City', 'Latitude', 'Longitude', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]
hotel_df = hotel_df[['City', 'Latitude', 'Longitude', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]
hotel_df = hotel_df.loc[(hotel_df["City"] != "abu samrah") & (hotel_df["City"] != "san borja") & 
                        (hotel_df["City"] != "baijiantan") ,:]
hotel_df = hotel_df.reset_index()
hotel_df.head(15)

,index,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,70,saint-ambroise,48.55,-71.33,77.41,34,40,6.93,CA,1598294836
1,190,huanta,-12.93,-74.25,75.20,23,20,5.82,PE,1598294875
2,246,espanola,35.99,-106.08,84.18,20,1,6.93,US,1598294894
3,270,annau,37.89,58.52,78.80,39,75,9.17,TM,1598294901
4,378,karratha,-20.74,116.85,75.51,37,0,7.14,AU,1598294932
5,427,lubbock,33.60,-101.83,88.81,31,20,9.17,US,1598294944
6,441,samarkand,39.65,66.96,75.20,38,0,6.93,UZ,1598294953
7,449,sabha,27.04,14.43,87.12,23,0,8.12,LY,1598294955
8,542,luz,-19.80,-45.69,76.55,30,0,7.20,BR,1598294985


In [116]:
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = { "key": g_key,
    "radius": 5000,
    "type": "lodging"
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    city = row["City"]
    # get lat, lng from df
    hotel_lat = row["Latitude"]
    hotel_lng = row["Longitude"]

#     # change location each iteration while leaving original params in place
    params["location"] = f"{hotel_lat},{hotel_lng}"
    print(f"Retrieving results for {index}: {city}")
#     # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#     # make request and print url
    response = requests.get(base_url, params=params).json()
    
# #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)

#     # convert to json
    results = response['results']

# #     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        row["Hotel Name"] = results[0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        row["Hotel Name"] = 'NaN'


Retrieving results for 0: saint-ambroise
Retrieving results for 1: huanta
Retrieving results for 2: espanola
Retrieving results for 3: annau
Retrieving results for 4: karratha
Retrieving results for 5: lubbock
Retrieving results for 6: samarkand
Retrieving results for 7: sabha
Retrieving results for 8: luz


In [117]:
hotel_df

,index,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,70,saint-ambroise,48.55,-71.33,77.41,34,40,6.93,CA,1598294836,
1,190,huanta,-12.93,-74.25,75.20,23,20,5.82,PE,1598294875,
2,246,espanola,35.99,-106.08,84.18,20,1,6.93,US,1598294894,
3,270,annau,37.89,58.52,78.80,39,75,9.17,TM,1598294901,
4,378,karratha,-20.74,116.85,75.51,37,0,7.14,AU,1598294932,
5,427,lubbock,33.60,-101.83,88.81,31,20,9.17,US,1598294944,
6,441,samarkand,39.65,66.96,75.20,38,0,6.93,UZ,1598294953,
7,449,sabha,27.04,14.43,87.12,23,0,8.12,LY,1598294955,
8,542,luz,-19.80,-45.69,76.55,30,0,7.20,BR,1598294985,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
